In [ ]:
#  Upload patient_data.csv
import pandas as pd
from google.colab import files
import io
from IPython.display import display

print(" Select and upload your patient_data.csv file (browser dialog)...")
uploaded = files.upload()


fname = next(iter(uploaded))
print("✅ Uploaded:", fname)


df = pd.read_csv(io.BytesIO(uploaded[fname]))


df.columns = [c.strip().lower() for c in df.columns]

print("Columns detected:", df.columns.tolist())
print("Preview of first rows:")
display(df.head())


 Select and upload your patient_data.csv file (browser dialog)...


Saving patient_data.csv to patient_data (1).csv
✅ Uploaded: patient_data (1).csv
Columns detected: ['age', 'gender', 'symptoms', 'diagnosis']
Preview of first rows:


,age,gender,symptoms,diagnosis
0,45,male,"Symptoms related to hypertension, uncontrolled...","hypertension, uncontrolled (for cerebrovascula..."
1,50,male,Symptoms related to ablation or mapping of car...,ablation or mapping of cardiac conduction pathway
2,83,male,"Symptoms related to endarterectomy, carotid","endarterectomy, carotid"
3,49,male,"Symptoms related to infarction, acute myocardi...","infarction, acute myocardial (mi)"
4,57,male,"Symptoms related to cabg alone, coronary arter...","cabg alone, coronary artery bypass grafting"


In [ ]:

required = ['age', 'sex', 'symptoms', 'diagnosis']

if 'gender' in df.columns and 'sex' not in df.columns:
    df.rename(columns={'gender': 'sex'}, inplace=True)
if 'age ' in df.columns:
    df.rename(columns={'age ': 'age'}, inplace=True)

df.columns = [c.strip().lower() for c in df.columns]

missing = [c for c in required if c not in df.columns]
if missing:
    print(" Missing required columns:", missing)
    print("Your CSV must contain these headers (case-insensitive):", required)
else:
    print("All required columns present ")
    print("Data preview:")
    display(df[required].head())


All required columns present 
Data preview:


,age,sex,symptoms,diagnosis
0,45,male,"Symptoms related to hypertension, uncontrolled...","hypertension, uncontrolled (for cerebrovascula..."
1,50,male,Symptoms related to ablation or mapping of car...,ablation or mapping of cardiac conduction pathway
2,83,male,"Symptoms related to endarterectomy, carotid","endarterectomy, carotid"
3,49,male,"Symptoms related to infarction, acute myocardi...","infarction, acute myocardial (mi)"
4,57,male,"Symptoms related to cabg alone, coronary arter...","cabg alone, coronary artery bypass grafting"


In [ ]:
# STEP 2: Generate Clinical Notes


!pip install transformers --quiet
from transformers import pipeline
from IPython.display import display


pipe = pipeline("text2text-generation", model="google/flan-t5-base")


def generate_clinical_note(patient):
    prompt = f"""
    You are a medical scribe. Convert the following raw patient data into
    a professional clinical note.

    Patient Data:
    Age: {patient['age']}
    Sex: {patient['sex']}
    Symptoms: {patient['symptoms']}
    Diagnosis: {patient['diagnosis']}

    Clinical Note:
    """
    output = pipe(prompt, max_new_tokens=150)[0]['generated_text']
    return output.strip()


clinical_notes = []

print("\n===== Generated Clinical Notes =====\n")
for idx, row in df.iterrows():
    note = generate_clinical_note(row)
    clinical_notes.append(note)
    print(f"--- Patient {idx+1} ---")
    print(note)
    print("\n")


df["clinical_note"] = clinical_notes

print(" Step 2 complete — Clinical notes added to dataframe")
display(df.head())


Device set to use cpu



===== Generated Clinical Notes =====

--- Patient 1 ---
Age: 45 Sex: male Symptoms: Symptoms related to hypertension, uncontrolled (for cerebrovascular accident-see neurological system) Diagnosis: hypertension, uncontrolled (for cerebrovascular accident-see neurological system)


--- Patient 2 ---
Age: 50 Sex: male Symptoms: Symptoms related to ablation or mapping of cardiac conduction pathway Diagnosis: ablation or mapping of cardiac conduction pathway


--- Patient 3 ---
Age: 83 Sex: male Symptoms: Symptoms related to endarterectomy, carotid Diagnosis: endarterectomy, carotid


--- Patient 4 ---
Symptoms: Symptoms related to infarction, acute myocardial (mi) Diagnosis: infarction, acute myocardial (mi)


--- Patient 5 ---
Age: 57 Sex: male Symptoms: Symptoms related to cabg alone, coronary artery bypass grafting Diagnosis: cabg alone, coronary artery bypass grafting


--- Patient 6 ---
Age: 80 Sex: female Symptoms: Symptoms related to emphysema/bronchitis Diagnosis: emphysema/bronch

,age,sex,symptoms,diagnosis,clinical_note
0,45,male,"Symptoms related to hypertension, uncontrolled...","hypertension, uncontrolled (for cerebrovascula...",Age: 45 Sex: male Symptoms: Symptoms related t...
1,50,male,Symptoms related to ablation or mapping of car...,ablation or mapping of cardiac conduction pathway,Age: 50 Sex: male Symptoms: Symptoms related t...
2,83,male,"Symptoms related to endarterectomy, carotid","endarterectomy, carotid",Age: 83 Sex: male Symptoms: Symptoms related t...
3,49,male,"Symptoms related to infarction, acute myocardi...","infarction, acute myocardial (mi)","Symptoms: Symptoms related to infarction, acut..."
4,57,male,"Symptoms related to cabg alone, coronary arter...","cabg alone, coronary artery bypass grafting",Age: 57 Sex: male Symptoms: Symptoms related t...


In [ ]:

# STEP 3: Map Diagnosis to ICD-10 Codes (Updated)


# ICD-10 dictionary
icd10_lookup = {
    "pneumonia": "J18.9",
    "hypertension": "I10",
    "ablation or mapping of cardiac conduction pathway": "Z98.89",
    "endarterectomy, carotid": "I70.0",
    "infarction, acute myocardial (mi)": "I21.9",
    "cabg alone, coronary artery bypass grafting": "Z95.1",
    "emphysema/bronchitis": "J44.9",
    "alcohol withdrawal": "F10.93",
    "overdose, sedatives, hypnotics, antipsychotics, benzodiazepines": "T43.501A",
    "diabetic ketoacidosis": "E10.10",
    "overdose, alcohols (bethanol, methanol, ethylene glycol)": "T51.0X1A",
    "cva, cerebrovascular accident/stroke": "I64",
    "renal failure, acute": "N17.9",
    "pneumonia, viral": "J12.9",
    "angina, unstable (angina interferes w/quality of life or meds are tolerated poorly)": "I20.0",
    "mi admitted > 24 hrs after onset of ischemia": "I21.4",
    "aortic valve replacement (isolated)": "Z95.2",
    "cabg with other operation": "Z95.1",
    "aortic and mitral valve replacement": "Z95.3",
    "aneurysm, thoracic aortic; with dissection": "I71.0",
    "chf, congestive heart failure": "I50.9",
    "sepsis, other": "A41.9",
    "rhythm disturbance (atrial, supraventricular)": "I49.9",
    "diabetes": "E11.9",
    "migraine": "G43.9",
    "asthma": "J45.9",
    "covid-19": "U07.1",
    "bronchitis": "J20.9",
    "heart failure": "I50.9",
    "chronic kidney disease": "N18.9",
    "anemia": "D64.9",
    "stroke": "I63.9",
    "influenza": "J10.1",
    "depression": "F32.9",
    "anxiety": "F41.9",
    "osteoarthritis": "M19.9",
    "gastroenteritis": "A09",
    "appendicitis": "K35.9",
    "epilepsy": "G40.9",
    "hyperthyroidism": "E05.9",
    "hypothyroidism": "E03.9"
}


def get_icd10_code(diagnosis):
    diagnosis_lower = diagnosis.lower()
    for key, code in icd10_lookup.items():
        if key.lower() in diagnosis_lower:
            return code

    return "R69"  # Illness, unspecified


df["icd10_code"] = df["diagnosis"].apply(get_icd10_code)


from IPython.display import display
print(" ICD-10 mapping complete")
display(df[["diagnosis", "icd10_code"]].head(20))


 ICD-10 mapping complete


,diagnosis,icd10_code
0,"hypertension, uncontrolled (for cerebrovascula...",I10
1,ablation or mapping of cardiac conduction pathway,Z98.89
2,"endarterectomy, carotid",I70.0
3,"infarction, acute myocardial (mi)",I21.9
4,"cabg alone, coronary artery bypass grafting",Z95.1
5,emphysema/bronchitis,J44.9
6,alcohol withdrawal,F10.93
7,"overdose, sedatives, hypnotics, antipsychotics...",T43.501A
8,diabetic ketoacidosis,E10.10
9,"overdose, alcohols (bethanol, methanol, ethyle...",T51.0X1A


In [ ]:

# STEP 4: Save results into CSV and JSON



output_csv = "patient_data_output.csv"
df.to_csv(output_csv, index=False)


output_json = "patient_data_output.json"
df.to_json(output_json, orient="records", indent=2)

print(f" Step 4 complete — Results saved as:")
print(f"    {output_csv}")
print(f"    {output_json}")


 Step 4 complete — Results saved as:
    patient_data_output.csv
    patient_data_output.json


In [ ]:
from google.colab import files
files.download(output_csv)
files.download(output_json)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>